In [72]:
import subprocess
import datetime
import os

import pandas as pd
from datetime import datetime





def gen_exec(cmd, check_return: bool = False):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line
    popen.stdout.close()
    return_code = popen.wait()
    if return_code and check_return:
        raise subprocess.CalledProcessError(return_code, cmd)


def exec_bash(cmd: str, check_return: bool = False):
    return [s for s in gen_exec(cmd.split(), check_return)]
    
def exec_bash_print(cmd: str, check_return: bool = False):
    for path in gen_exec(cmd.split(), check_return):
        print(path, end="")
        
def strtime():
    return datetime.now().strftime("%y%m%d_%H%M%S")


exec_bash_print("ls")

Untitled.ipynb


In [ ]:
#cmd= f"java -cp {conf.jar_file} {javafile} {cmd}"
#output = exec_bash(cmd)

In [28]:
from pathlib import Path

prj_path = Path(str(Path("../../../").resolve())+"/")
exp_folder = Path(prj_path, "papers/neurips21/")
res_folder = Path(exp_folder, "output")
model_folder = Path(exp_folder, "models")

jar_file = Path(prj_path, "target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar")
javafile = Path(exp_folder, "Experiments.java")
java = "/Library/Java/JavaVirtualMachines/openjdk-12.0.1.jdk/Contents/Home/bin/java"

print(prj_path)
print(exp_folder)
print(res_folder)
print(model_folder)

print(jar_file)



/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/papers/neurips21
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/papers/neurips21/output
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/papers/neurips21/models
/Users/rcabanas/GoogleDrive/IDSIA/causality/dev/credici/target/credici-0.1.3-SNAPSHOT-jar-with-dependencies.jar


In [73]:
def run(model, datasize = 1000, executions=20, logfile=None):
    logfile = logfile or Path(res_folder, f"{strtime()}_log.txt")
    print(model)
    modelfile = Path(model_folder, model)
    params = f"--executions {executions} --datasize {datasize} --selectpol LAST --output {res_folder} "\
        f"--logfile {logfile} -q {modelfile}"

    cmd = f"{java} -cp {jar_file} {javafile} {params}"
    output = exec_bash(cmd)  
    #print(output)
    exec(output[0])
    return locals()["results"]

results = run(model = "set1/chain_twExo1_nEndo4_6.uai", datasize=500)

In [69]:
results["pnsEM"]

[[0.007467470035376679, 0.007467470035376679],
 [0.007467470035376679, 0.014968245923306627],
 [0.007467470035376679, 0.02475019532233444],
 [0.0012109142126740784, 0.02475019532233444],
 [1.1363096522883869e-05, 0.02475019532233444],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028098539901715097],
 [1.1363096522883869e-05, 0.028351153414640912],
 [1.1363096522883869e-05, 0.028351153414640912],
 [1.1363096522883869e-05, 0.028351153414640912],
 [1.1363096522883869e-05, 0.028351153414640912],
 [1.1363096522883869e-05, 0.028351153414640912],
 [1.1363096522883869e-05, 0.028351153414640912]]

In [85]:
modelset="set1"
datasize = 1000


logfile = logfile or Path(res_folder, f"{strtime()}_log.txt")
models = [Path(modelset, file) for file in os.listdir(Path(model_folder,modelset)) if file.endswith(".uai")]
results = dict()
print(logfile)
for m in models[0:4]:
    results[str(m)] = run(model = m, datasize=datasize, logfile=logfile)
    


In [89]:
len(results)

4

In [100]:
list(results.values())[4]

IndexError: list index out of range

40